In [2]:
import nb_setup
import importlib

importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [10]:
from utils.fetch_coin import fetch_coin
df = fetch_coin(
    coin="BTC",
    interval="4h",
    bar_count=380,
)
df.shape

BTC


(381, 13)

In [31]:
import pandas as pd
import numpy as np
from enum import Enum
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def _plotly_strategy(data):
    """
    Use Plotly to create an interactive plot of OHLCV data with Bollinger Bands
    and overlays of buy/sell markers based on the history DataFrame.
    """
    # fix data
    data = data.copy()
    if not isinstance(data.index, pd.DatetimeIndex):
        data.set_index("datetime", inplace=True)

    # Create the figure
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.02,
        subplot_titles=("Candlestick", "Volumn", "-"),
        row_width=[
            # 0.2,
            0.3,
            0.7,
        ],
    )

    # Add Bollinger Bands
    # Create the Candlestick chart +
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data["open"],
            high=data["high"],
            low=data["low"],
            close=data["close"],
            name="OHLC",
        ),
        row=1,
        col=1,
    )
    # EMA 200
    if "ema200" in data.columns:
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data["ema200"],
                line=dict(color="blue", width=1),
                name="EMA 200",
            ),
            row=1,
            col=1,
        )

    ########### Graph 2 ###########
    # Volume bars
    # fig.add_trace(
    #     go.Bar(x=data.index, y=data["volume"], name="Volume", marker_color="blue"),
    #     row=2,
    #     col=1,
    # )
    # Area
    if "area" in data.columns:
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data["area"],
                line=dict(color="black", width=1),
                name="Area",
            ),
            row=1,
            col=1,
        )
    # normalized area
    if "sum_norm_area" in data.columns:
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data["sum_norm_area"],
                line=dict(color="blue", width=1),
                name="Norm Area",
            ),
            row=2,
            col=1,
        )

    # cash

    # Update layout for a more detailed view
    fig.update_layout(
        title="Bollinger Bands Strategy Plot",
        yaxis_title="Price",
        xaxis_title="Date",
        # width=1300,  # Set the width of the plot
        height=1000,  # Set the height of the plot
        xaxis_rangeslider_visible=False,
    )  # Disable range slider for more space

    # Make x-axis range sliders visible only for the bottom plot
    fig.update_xaxes(
        rangeslider_visible=True, row=2, col=1
        )

    fig.show()



In [35]:
_df = df.copy()

# calculate EMA of 200
_df["ema200"] = _df["close"].ewm(span=200, adjust=False).mean()

# calcualate area under curve for the laster 180 candles
# if close > ema200, then area = close - ema200
# if close < ema200, then area = close - ema200 [ negative number ]
_df["area"] = _df["close"] - _df["ema200"]
# normalize the area base on the mean of the ema200 for the last 180 candles
_df["norm_area"] = _df["area"] / _df["ema200"].mean()
# sum the area if the last 180 candles, only need the last 180 candles
_df["sum_norm_area"] = _df["norm_area"].rolling(180).sum()
print(_df["sum_norm_area"].values[-1])

3.3637013305979995


In [36]:
_plotly_strategy(_df)